In [19]:
# Monta Google Drive per accedere ai file
from google.colab import drive
drive.mount('/content/drive')

# Importa le librerie necessarie
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense

# Carica il dataset da Google Drive
file_path = "/content/drive/MyDrive/cyber/definitivo.csv"
df = pd.read_csv(file_path)

# Rimuovi spazi bianchi dai nomi delle colonne
df.columns = df.columns.str.strip()

# Converti la colonna 'Normal/Attack' in numeri
df['Normal/Attack'] = df['Normal/Attack'].map({'Normal': 0, 'Attack': 1})

# Rimuovi la colonna 'Timestamp' poiché non è necessaria per il rilevamento delle anomalie
df = df.drop(columns=['Timestamp'])

# Rimuovi tutte le righe che contengono almeno un valore NaN
df.dropna(inplace=True)

# Dividi il dataset in features (X) e target (y)
X = df.drop(columns=['Normal/Attack'])
y = df['Normal/Attack']

# Standardizza le features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividi il dataset in training set e test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Reshape dei dati di input per includere le informazioni temporali
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Crea il modello
def create_lstm_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(32, return_sequences=False),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model
# Imposta la forma dei dati correttamente per l'input del modello LSTM
input_shape = (X_train.shape[1], 1)

# Crea il modello
model = create_lstm_model(input_shape)

# Fai il fit del modello
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.2)

# Valuta il modello sul set di test
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/10
6308/6308 [==============================] - 67s 10ms/step - loss: 0.0693 - accuracy: 0.9864 - val_loss: 0.0463 - val_accuracy: 0.9899
Epoch 2/10
6308/6308 [==============================] - 59s 9ms/step - loss: 0.0361 - accuracy: 0.9915 - val_loss: 0.0375 - val_accuracy: 0.9914
Epoch 3/10
6308/6308 [==============================] - 59s 9ms/step - loss: 0.0218 - accuracy: 0.9943 - val_loss: 0.0266 - val_accuracy: 0.9940
Epoch 4/10
6308/6308 [==============================] - 59s 9ms/step - loss: 0.0171 - accuracy: 0.9953 - val_loss: 0.0139 - val_accuracy: 0.9959
Epoch 5/10
6308/6308 [==============================] - 57s 9ms/step - loss: 0.0147 - accuracy: 0.9958 - val_loss: 0.0128 - val_accuracy: 0.9962
Epoch 6/10
6308/6308 [==============================] - 57s 9ms/step - loss: 0.0131 - accuracy: 0.9962 - val_loss: 0.0117 - val_accuracy: 0.9965


In [20]:
# Effettua previsioni sul set di test
y_pred = model.predict(X_test)

# Imposta una soglia per classificare le previsioni come anomalie
threshold = 0.5
# Confronta le previsioni con la soglia per identificare gli esempi anomali
y_pred_anomalies = (y_pred > threshold).astype(int)

# Valuta le prestazioni del modello
print(classification_report(y_test, y_pred_anomalies))

13516/13516 [==============================] - 47s 3ms/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    416373
         1.0       0.98      0.95      0.96     16132

    accuracy                           1.00    432505
   macro avg       0.99      0.97      0.98    432505
weighted avg       1.00      1.00      1.00    432505



Precision: Rappresenta la proporzione di istanze classificate come positive che sono effettivamente positive. In altre parole, è la capacità del modello di non etichettare erroneamente le istanze negative come positive. Per la classe 0 (non-anomalia), la precisione è del 100%, il che significa che tutte le istanze classificate come non-anomalie sono effettivamente non-anomalie. Per la classe 1 (anomalia), la precisione è del 98%, il che indica che il 2% delle istanze classificate come anomalie potrebbe essere un falso positivo.

Recall: Rappresenta la proporzione di istanze positive totali che sono state identificate correttamente dal modello. In altre parole, è la capacità del modello di trovare tutte le istanze positive. Per la classe 0, il recall è del 100%, il che indica che tutte le istanze di non-anomalie sono state identificate correttamente. Per la classe 1, il recall è del 95%, il che significa che il 5% delle istanze di anomalie potrebbe non essere stato rilevato dal modello.

F1-score: È una media ponderata della precisione e del recall. Rappresenta l'equilibrio tra precisione e recall. Un punteggio F1 alto indica un buon equilibrio tra la precisione e il recall.

Support: Rappresenta il numero di istanze in ciascuna classe nel set di dati di test.

Accuracy: Rappresenta la proporzione di predizioni corrette rispetto al totale delle predizioni effettuate dal modello. Nel tuo caso, l'accuracy è del 100%, il che significa che tutte le predizioni fatte dal modello sono corrette nel set di dati di test.